In [8]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from glob import glob
import os
import time
from cStringIO import StringIO

import numpy as np
import scipy.linalg
import scipy.stats

import emcee

import kepler
import threebody

In [10]:
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 120

In [11]:
jobid = "4375"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]
d = np.load(os.path.join(job_dir,"fitted_parameters.npz"))
parameter_means = d['parameter_means'][None][0]
parameter_errors = d['parameter_errors'][None][0]
fitter_params = d['fitter_params'][None][0]
parameter_formatted = dict((p,threebody.fmt(parameter_means[p],parameter_errors[p])) for p in parameter_means)

In [12]:
F = threebody.Fitter(**fitter_params)

In [13]:
mean_parameters = F.best_parameters.copy()
for p in F.best_parameters:
    if p in parameter_means:
        mean_parameters[p] = parameter_means[p]
for p in F.best_parameters:
    print p, F.best_parameters[p], mean_parameters[p]

eps1_o 0.0351859658148 0.0351860224301
eps1_i 0.000688262063085 0.000688040813265
asini_i 1.21752527797 1.21752555166
eps2_i -9.13954168319e-05 -9.13888774764e-05
delta_lan 2.19913601188e-06 6.18993947387e-06
eps2_o -0.00346306969577 -0.00346288427579
q_i 0.137405570412 0.13740968433
asini_o 74.672633135 74.6726478696
pb_o 327.256885572 327.257012362
delta -1.08530037771e-06 -8.6798823797e-07
pb_i 1.62939341669 1.62939413212
dgamma -1.33795394689e-05 5.3041936512e-07
tasc_i 0.40771664137 0.407716710232
acosi_o 91.361037238 91.3743573428
dbeta 0.00152960556651 -0.00129248335819
acosi_i 1.48955415134 1.48969547328
tasc_o 313.935159977 313.935286772


In [14]:
mean_parameters = F.best_parameters.copy()
for p in F.best_parameters:
    if p in parameter_means:
        mean_parameters[p] = parameter_means[p]
for p in F.best_parameters:
    print p, F.best_parameters[p], mean_parameters[p]

eps1_o 0.0351859658148 0.0351860224301
eps1_i 0.000688262063085 0.000688040813265
asini_i 1.21752527797 1.21752555166
eps2_i -9.13954168319e-05 -9.13888774764e-05
delta_lan 2.19913601188e-06 6.18993947387e-06
eps2_o -0.00346306969577 -0.00346288427579
q_i 0.137405570412 0.13740968433
asini_o 74.672633135 74.6726478696
pb_o 327.256885572 327.257012362
delta -1.08530037771e-06 -8.6798823797e-07
pb_i 1.62939341669 1.62939413212
dgamma -1.33795394689e-05 5.3041936512e-07
tasc_i 0.40771664137 0.407716710232
acosi_o 91.361037238 91.3743573428
dbeta 0.00152960556651 -0.00129248335819
acosi_i 1.48955415134 1.48969547328
tasc_o 313.935159977 313.935286772


In [15]:
resids = F.residuals(mean_parameters)

In [16]:
rms_resid_us = np.sqrt(np.mean(resids**2))/parameter_means['f0'] * 1e6
wts = F.phase_uncerts**(-2)
rms_resid_us_wt = np.sqrt(np.sum(wts*resids**2)/np.sum(wts))/parameter_means['f0'] * 1e6
print rms_resid_us, rms_resid_us_wt

1.54387714432 0.76230403208


In [17]:
2./(360*60*60*1000)*24*60*60

0.0001333333333333333

In [18]:
def fmt_tex(x, u):
    x = float(x)
    u = float(u)
    if u<0:
        raise ValueError("Uncertainty %g < 0" % u)
    elif u==0:
        return "%g" % x
    exponent_number = np.floor(np.log10(np.abs(x)))
    exponent_error = np.floor(np.log10(u))
    if u*10**(-exponent_error)>=2:
        ndigs = 1
    else:
        ndigs = 2
    if -1<=exponent_number<=2:
        no_exponent = True
    else:
        no_exponent = False
    if exponent_error>exponent_number:
        fstr = "$0({:"+str(ndigs)+r"d})\times 10^{{ {:d} }}$"
        return fstr.format(
            int(np.rint(u*10**(-exponent_error+(ndigs-1)))),
            int(exponent_error-ndigs+1))
    
    number_digits = int(exponent_number-exponent_error)+ndigs-1
    if no_exponent:
        fstr = "${:."+str(number_digits-int(exponent_number))+r"f}({:d})$"
        return fstr.format(
           x,
           int(np.rint(u*10**(-exponent_error+(ndigs-1)))))
    else:
        fstr = "${:."+str(number_digits)+r"f}({:d})\times 10^{{ {:d} }}$"
        return fstr.format(
           x*10**(-exponent_number),
           int(np.rint(u*10**(-exponent_error+(ndigs-1)))),
           int(exponent_number))

def fmt_mjd(x, u, base_mjd):
    x = float(x)
    u = float(u)
    if u<0:
        raise ValueError("Uncertainty %g < 0" % u)
    elif u==0:
        raise ValueError("Uncertainty %g == 0" % u)
        #return "%g" % x
    fracpart = (x-np.floor(x)) + (base_mjd-np.floor(base_mjd))
    intpart = np.floor(x)+np.floor(base_mjd)+np.floor(fracpart)
    fracpart -= np.floor(fracpart)
    exponent_error = np.floor(np.log10(u))
    if u*10**(-exponent_error)>=2:
        ndigs = 1
    else:
        ndigs = 2
    number_digits = int(-exponent_error)+ndigs-1
    fstr = "${:5d}.{:d}({:d})$"
    return fstr.format(int(intpart),
       int(np.rint(fracpart*10**number_digits)),
       int(np.rint(u*10**(-exponent_error+(ndigs-1)))))

print fmt_tex(1000,0.013)
print fmt_mjd(531.27,0.03,55920)

$1.000000(13)\times 10^{ 3 }$
$56451.27(3)$


In [31]:
class TableEntry:
    def __init__(self, description, mathname, value, error, units, base_mjd=None):
        self.description = description
        self.mathname = mathname
        self.value = value
        self.error = error
        self.units = units
        self.base_mjd = base_mjd
        
    def as_tex(self):
        if self.base_mjd is None:
            return (self.description + " & " +
                    self.mathname + " & " +
                    fmt_tex(self.value, self.error) + " " + self.units + r"""\\
""")
        else:
            return (self.description + " & " +
                    self.mathname + " & MJD " +
                    fmt_mjd(self.value, self.error, self.base_mjd) + r"""\\
""")

In [32]:
class TableEntryString:
    def __init__(self, description, mathname, value):
        self.description = description
        self.mathname = mathname
        self.value = value
        
    def as_tex(self):
        return (self.description + " & " +
                self.mathname + " & " +
                self.value + r"""\\
""")

In [50]:
s = r"""\hline
\multicolumn{3}{c}{%s}\\
\hline
"""

In [51]:
s2 = r"""\multicolumn{3}{c}{%s}\\""" + "\n"


In [52]:
class Divider:
    def __init__(self, kind, text=None):
        self.kind = kind
        self.text = text
    def as_tex(self):
        if self.kind=="h":
            return s % self.text
        elif self.kind==" ":
            return s2 % self.text
        else:
            raise ValueError

In [53]:
def format_table(entries):
    o = StringIO()
    o.write(r"""\begin{tabular}{lcc}
Parameter & Symbol & Value \\
""")
    for e in entries:
        o.write(e.as_tex())
    o.write(r"""\hline
\end{tabular}
""")
    return o.getvalue()

In [69]:
delta_systematic = 8e-7

table = format_table([
    Divider("h","Fixed values"),
    TableEntryString("Right ascension",
        r"RA",r"$03^h 37^m 43^s.82589(13)$"),
    TableEntryString("Declination",
        r"Dec",r"$+17\degree 15' 14'' .828(2)$"),
    TableEntryString("Dispersion measure",
        r"DM",r"$21.3162(3)$\,pc\,cm$^{-3}$"),
    TableEntryString("Solar system ephemeris",
        r"","DE435"),
    TableEntryString("Reference epoch",
        r"","MJD %d.0" % F.base_mjd),
    TableEntryString("Observation span",
        r"","MJD %.1f--%.1f" % (F.mjds[0]+F.base_mjd,F.mjds[-1]+F.base_mjd)),
    TableEntryString("Number of TOAs",
        r"","%d" % len(F.mjds)),
    TableEntryString("Root-mean-squared residual (unweighted)",
        r"",r"$%.1f\,\mu$s" % rms_resid_us),
    Divider("h","Fitted parameters"),
    Divider(" ","Spin-down parameters"),
    #TableEntry("Zero of pulsar phase",
    #    r"$t_0$",parameter_means["tzrmjd"],parameter_errors["tzrmjd"],"MJD",base_mjd=F.base_mjd),
    TableEntry("Pulsar spin frequency",
        r"$f$",parameter_means["f0"],parameter_errors["f0"],"Hz"),
    TableEntry("Spin frequency derivative",
        r"$\dot f$",parameter_means["f1"],parameter_errors["f1"],"Hz s$^{-1}$"),
    Divider(" ","Inner Keplerian parameters for pulsar orbit"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_I$",parameter_means["asini_i"],parameter_errors["asini_i"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,I}$",parameter_means["pb_i"],parameter_errors["pb_i"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \Omega$)",
        r"$\epsilon_{1,I}$",parameter_means["eps1_i"],parameter_errors["eps1_i"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,I}$",parameter_means["eps2_i"],parameter_errors["eps2_i"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},I}$",parameter_means["tasc_i"],parameter_errors["tasc_i"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Outer Keplerian parameters for centre of mass of inner binary"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_O$",parameter_means["asini_o"],parameter_errors["asini_o"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,O}$",parameter_means["pb_o"],parameter_errors["pb_o"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \Omega$)",
        r"$\epsilon_{1,O}$",parameter_means["eps1_o"],parameter_errors["eps1_o"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,O}$",parameter_means["eps2_o"],parameter_errors["eps2_o"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},O}$",parameter_means["tasc_o"],parameter_errors["tasc_o"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Interaction parameters"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_I$",parameter_means["acosi_i"],parameter_errors["acosi_i"],"lt-s"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_O$",parameter_means["acosi_o"],parameter_errors["acosi_o"],"lt-s"),
    TableEntry("Inner companion mass over PSR mass",
        r"$q_I = m_{cI}/m_p$",parameter_means["q_i"],parameter_errors["q_i"],""),
    TableEntry("Difference in longs. of asc. nodes",
        r"$\delta_{\text{lan}}$",parameter_means["delta_lan_deg"],parameter_errors["delta_lan_deg"],"$\degree$"),
    Divider(" ","GR violation parameters"),
    TableEntry(r"PPN nonlinearity-of-gravity parameter",
        r"$\beta_{\text{PPN}}-1$",parameter_means["dbeta"],parameter_errors["dbeta"],""),
    TableEntry(r"PPN space curvature parameter",
        r"$\gamma_{\text{PPN}}-1$",parameter_means["dgamma"],parameter_errors["dgamma"],""),
    # FIXME: handle systematic errors better
    #TableEntry(r"SEP violation parameter (see text)",
    #    r"$\Delta$",parameter_means["delta"],parameter_errors["delta"],""),
    TableEntryString(r"SEP violation parameter",
        r"$\Delta$",
        r"$(%.1f \pm %.1f \pm %.1f) \times 10^{-6}$"
                     % (1e6*parameter_means["delta"],
                        1e6*parameter_errors["delta"],
                        1e6*delta_systematic)),
    Divider("h","Inferred values"),
    Divider(" ","Pulsar properties"),
    TableEntry("Pulsar period",
        r"$P$",parameter_means["P"]*1e3,parameter_errors["P"]*1e3,"ms"),
    TableEntry("Pulsar period derivative",
        r"$\dot P$",parameter_means["Pdot"],parameter_errors["Pdot"],""),
    #TableEntry("Inferred surface dipole magnetic field",
    #    r"$B$",parameter_means["B"],parameter_errors["B"],"G"),
    #TableEntry("Spin-down power",
    #    r"$\dot E$",parameter_means["Edot"],parameter_errors["Edot"],"erg s$^{-1}$"),
    #TableEntry("Characteristic age",
    #    r"$\tau$",parameter_means["tau"],parameter_errors["tau"],"y"),
    TableEntryString("Inferred surface dipole magnetic field",
        r"$B$",r"$1.6\times 10^8$ G"),
    TableEntryString("Spin-down power",
        r"$\dot E$",r"$1.7\times 10^{34}$ erg s$^{-1}$"),
    TableEntryString("Characteristic age",
        r"$\tau$",r"$4.9\times 10^9$ y"),
    Divider(" ","Orbital geometry"),
    TableEntry("Pulsar semimajor axis (inner)",
        r"$a_I$",parameter_means["a_i"],parameter_errors["a_i"],"lt-s"),
    TableEntry("Eccentricity (inner)",
        r"$e_I$",parameter_means["e_i"],parameter_errors["e_i"],""),
    TableEntry("Longitude of periastron (inner)",
        r"$\omega_I$",parameter_means["omega_i_deg"],parameter_errors["omega_i_deg"],"$\degree$"),
    TableEntry("Pulsar semimajor axis (outer)",
        r"$a_O$",parameter_means["a_o"],parameter_errors["a_o"],"lt-s"),
    TableEntry("Eccentricity (outer)",
        r"$e_O$",parameter_means["e_o"],parameter_errors["e_o"],""),
    TableEntry("Longitude of periastron (outer)",
        r"$\omega_O$",parameter_means["omega_o_deg"],parameter_errors["omega_o_deg"],"$\degree$"),
    TableEntry("Inclination of invariant plane",
        r"$i$",parameter_means["i_deg"],parameter_errors["i_deg"],"$\degree$"),
    TableEntry("Inclination of inner orbit",
        r"$i_I$",parameter_means["i_i_deg"],parameter_errors["i_i_deg"],"$\degree$"),
    TableEntry("Angle between orbital planes",
        r"$\delta_i$",parameter_means["delta_i_deg"],parameter_errors["delta_i_deg"],"$\degree$"),
    TableEntry("Angle between eccentricity vectors",
        r"$\delta_\omega \sim \omega_O-\omega_I$",parameter_means["delta_omega_deg"],parameter_errors["delta_omega_deg"],"$\degree$"),
    Divider(" ","Masses"),
    TableEntry("Pulsar mass",
        r"$m_p$",parameter_means["m_psr"],parameter_errors["m_psr"],"$M_\Sun$"),
    TableEntry("Inner companion mass",
        r"$m_{cI}$",parameter_means["m_1"],parameter_errors["m_1"],"$M_\Sun$"),
    TableEntry("Outer companion mass",
        r"$m_{cO}$",parameter_means["m_2"],parameter_errors["m_2"],"$M_\Sun$"),
    ])
print table

\begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
\multicolumn{3}{c}{Fixed values}\\
\hline
Right ascension & RA & $03^h 37^m 43^s.82589(13)$\\
Declination & Dec & $+17\degree 15' 14'' .828(2)$\\
Dispersion measure & DM & $21.3162(3)$\,pc\,cm$^{-3}$\\
Solar system ephemeris &  & DE435\\
Reference epoch &  & MJD 55920.0\\
Observation span &  & MJD 55956.7--57866.9\\
Number of TOAs &  & 27110\\
Root-mean-squared residual (unweighted) &  & $1.5\,\mu$s\\
\hline
\multicolumn{3}{c}{Fitted parameters}\\
\hline
\multicolumn{3}{c}{Spin-down parameters}\\
Pulsar spin frequency & $f$ & $365.953363083(4)$ Hz\\
Spin frequency derivative & $\dot f$ & $-1.1775454(13)\times 10^{ -15 }$ Hz s$^{-1}$\\
\multicolumn{3}{c}{Inner Keplerian parameters for pulsar orbit}\\
Semimajor axis projected along line of sight & $(a \sin i)_I$ & $1.2175256(2)$ lt-s\\
Orbital period & $P_{b,I}$ & $1.6293941(7)$ d\\
Eccentricity parameter ($e\sin \Omega$) & $\epsilon_{1,I}$ & $6.880(2)\times 10^{ -4 }$ \\
Eccentri

In [72]:
with open("table.tex", "wt") as f:
    f.write(r"""%\documentclass{article}
%\usepackage{amsmath}
%\usepackage{gensymb}
%\newcommand{\Sun}{\odot}
%\begin{document}
% DO NOT EDIT - automatically generated
{\scriptsize
""")
    f.write(table)
    f.write(r"""
}

The system parameters for PSR~J0337+17. Values in parentheses represent 
$1\sigma$ errors in the last decimal place(s), as determined by our MCMC 
fitting. In the top section are parameter 
values we supplied as input to our timing fit; the position was determined 
with VLBI, and the DM we determined by examining a single observation. 
In the middle section are
the parameters we used to describe the state of the system as of the
reference epoch, which is when we began our differential equation 
integrations. Along with the pulsar spin-down parameters, these parameters 
are further divided into the conventional Keplerian parameters measurable 
in binary pulsars, one set for each of the orbits, plus four parameters 
that are measurable only because of orbital interactions. These fourteen 
parameters plus the longitude of the ascending node of the inner system 
and the center of mass position and velocity, all assumed to be zero, can 
completely describe any combination of three positions, velocities, and 
masses. 
The last subsection contains parameters that describe the degree to which 
this timing solution differs from GR. The PPN parameters $\beta$ and $\gamma$
are not substantially constrained by our observations, and their values and 
uncertainties are consistent with the priors we obtained from Solar System
experiments. The SEP vioaltion parameter $\Delta$ is the fractional difference
in acceleration between the pulsar and the inner white dwarf, and because it is
the focus of this paper we take additional steps to estimate the impact of 
systematics on it.
The fourth section contains quantities we inferred from the result
of our fitting. Although some of the fitting parameters are highly covariant,
we computed inferred parameters and their errors based on a sample from the
posterior distribution, correctly taking into account these covariances. 
Note that since we do not correct for the mean slowing, the pulsar spin 
frequency and its derivative are changed by a fractional amount on the 
order of $10^{-6}$, and therefore so are all quantities derived from them.
We use the standard formulae for computing $B$, $\dot E$, and $\tau$; in 
particular we assume a pulsar mass of $1.4M_\Sun$ and a moment of inertia
of $10^{45}\;g\;cm^2$.
The Laplace-Lagrange parameters $\epsilon_1$ and $\epsilon_2$ provide a 
parameterization of the eccentricity ($e$) and longitude of periastron ($\Omega$) 
of an orbit that avoids a coordinate singularity at zero eccentricity; 
the pair $(\epsilon_2,\epsilon_1)$ forms a vector in the plane of the orbit 
called the eccentricity vector.
For a single orbit, the ascending node is the place where the pulsar passes 
through the plane of the sky moving away from us; the longitude of the 
ascending node specified the orientation of the orbit on the sky. This is not 
measurable with the data we have, but the difference between the longitudes of
the ascending nodes of the two orbits is measurable through orbital interactions.
The invariant plane is the plane perpendicular to the total (orbital) angular
momentum of the triple system.
Masses are as measured by a distant observer; corrections for special 
relativity are at the $10^{-8}$ fractional level and are therefore irrelevant.

%\end{document}""")


In [84]:
"""
The zero of pulsar phase is to some degree arbitrary; it is based on 
alignment with our reference profile for the Westerbork 1400~MHz 
observations. It is also measured at the solar system barycenter and at 
infinite frequency.
"""

In [7]:
np.average?

In [56]:
parameter_means

{'B': 156855661.73257729555,
 'Edot': 1.702160499718938014e+34,
 'P': 0.0027325886325366150635,
 'Pdot': 8.7927795443249636513e-21,
 'a_i': 1.9239441482798994213,
 'a_o': 118.00541333636190835,
 'a_tot_i': 15.925462006012838122,
 'a_tot_o': 588.48090158648086406,
 'acosi_i': 1.4896954732838410983,
 'acosi_o': 91.37435734279643812,
 'asini_i': 1.2175255516555931047,
 'asini_o': 74.672647869588037725,
 'dbeta': -0.001292483358193196038,
 'delta': -8.6798823796983757654e-07,
 'delta_i': 4.9098274332476952e-05,
 'delta_i_deg': 0.0028131239006264287,
 'delta_lan': 6.1899394738712141069e-06,
 'delta_lan_deg': 0.00035465740729424986927,
 'delta_omega': -0.033951284457028044438,
 'delta_omega_deg': -1.9452653084358176568,
 'dgamma': 5.3041936511959916593e-07,
 'e_i': 0.00069408363178287931598,
 'e_o': 0.035356014226554930815,
 'eps1_i': 0.00068804081326490438394,
 'eps1_o': 0.035186022430071315914,
 'eps2_i': -9.1388877476418056856e-05,
 'eps2_o': -0.0034628842757855533136,
 'f0': 365.95336308